К настоящему моменту мы достигли неплохих результатов. Например, имея на руках модель камеры, можем легко превратить с её помощью трехмерную сцену известной структуры в плоское изображение. Для решения задач в области компьютерной графики, этих знаний вполне достаточно: компьютер позволяет програмно задать как структуру сцены, так и модель камеры. 

На практике же полная информация о сцене и камере обычно недоступна. Всё, на что мы можем рассчитывать &ndash; это набор изображений. Цель следующих глав &ndash; разобраться в том, какую информацию и каким образом из этих изображений можно извлечь.

# Калибровка камеры

Сперва научимся самостоятельно определять параметры камеры (то есть строить её модель), этот процесс называется калибровкой. В первом приближении мы ослабим условие задачи и будем считать, что структура сцены известна. Мы точно знаем, как расположены объекты относительно некоторой мировой системы координат. Камера же просто перемещается вокруг и порождает изображения согласно свой внутренней механике. 



Реальность не позволяет точно задать положения всех объектов сцены. Поэтому во втором приближении придется прибегнуть к некоторому трюку, а именно сконцентрировать внимание на специальных объектах, информация о геометрии которых заранее известна. В качестве таких объектов выступают калибровочные шаблоны.

<img src="../assets/images/calibration_pattern.png" alt="calibration pattern" style="background-color:#d3d3d3"/> 
</br>

In [ ]:
# Задача на определение внутренних параметров
# Задача на определение внешних параметров